In [1]:
#Importing json data
#Fetching the JSON data through the API link 

import requests
import pandas as pd
from requests.api import head
import json

#fetching json data
response = requests.get('https://data.wa.gov/api/views/f6w7-q2d2/rows.json?accessType=DOWNLOAD')
json_data = response.json()

columns = json_data['meta']['view']['columns']
columns_to_exclude = 8
print(columns) 

[{'id': -1, 'name': 'sid', 'dataTypeName': 'meta_data', 'fieldName': ':sid', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']}, {'id': -1, 'name': 'id', 'dataTypeName': 'meta_data', 'fieldName': ':id', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']}, {'id': -1, 'name': 'position', 'dataTypeName': 'meta_data', 'fieldName': ':position', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']}, {'id': -1, 'name': 'created_at', 'dataTypeName': 'meta_data', 'fieldName': ':created_at', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']}, {'id': -1, 'name': 'created_meta', 'dataTypeName': 'meta_data', 'fieldName': ':created_meta', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']}, {'id': -1, 'name': 'updated_at', 'dataTypeName': 'meta_data', 'fieldName': ':updated_at', 'position': 0, 'renderTypeName': 'meta_data', 'format': {}, 'flags': ['hidden']

In [3]:
#Converting json data fetched through API to csv file 

import requests
import pandas as pd
from requests.api import head
import json
import csv

#fetching json data
response = requests.get('https://data.wa.gov/api/views/f6w7-q2d2/rows.json?accessType=DOWNLOAD')
json_data = response.json()

columns = json_data['meta']['view']['columns']
columns_to_exclude = 8
#print(columns)

rows = json_data['data']

# Extract column names from metadata
with open('Vehdata.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [col['name'] for col in columns][columns_to_exclude:]  
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)

    # Write rows to the CSV file
    for row in rows:
        writer.writerow(row[columns_to_exclude:])
        
        
#The output of the above code will be a csv file named Vehdata

In [13]:
#Converting null values to NA

import pandas as pd
import csv


csv_data = pd.read_csv('Vehdata.csv')
csv_data.fillna('NA', inplace=True)
csv_data.to_csv('ElectricVehdata.csv', index=False)

In [4]:
#Creating table for csv file using mysql 
#Installing Packages

%pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 7.9/7.9 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 904.2/904.2 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
#Importing the csv file into mysql to create a table

import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root',
    password='Caroline@5')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE X22114386Project")
        print("X22114386Project  database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

X22114386Project  database is created


In [10]:
#Creating new table name ElectricVehicleData to add the csv file 

import mysql.connector
import csv

# Connect to the MySQL database
conn = mysql.connector.connect( host="localhost",database="X22114386Project",user="root",password="Caroline@5")

if conn.is_connected():
 cursor = conn.cursor()
 cursor.execute("select database();")
 record = cursor.fetchone()
 print("You're connected to database.")
 cursor.execute('DROP TABLE IF EXISTS ElectricVehicleData;')  
 print('Creating table....')

 #creating table by inserting all the column names
 cursor.execute("CREATE TABLE ElectricVehicleData (VIN CHAR(50) NOT NULL, County CHAR(30) NOT NULL, City CHAR(30) NOT NULL, State CHAR(50) NOT NULL, Postal_Code CHAR(50) NOT NULL, Model_Year CHAR(30) NOT NULL, Make CHAR(50) NOT NULL, Model CHAR(50) NOT NULL, Electric_Vehicle_Type CHAR(100) NOT NULL, Clean_Alternative_Fuel_Vehicle_Eligibility CHAR(100) NOT NULL, Electric_Range CHAR(50) NOT NULL, Base_MSRP CHAR(50) NOT NULL, Legislative_District CHAR(50) NOT NULL, DOL_Vehicle_ID CHAR(50) NOT NULL, Vehicle_Location CHAR(50) NOT NULL, Electric_Utility CHAR(250) NOT NULL, Census_Tract CHAR(50) NOT NULL, Counties CHAR(50) NOT NULL, Congressional_Districts CHAR(50) NOT NULL, WAOFM_GIS CHAR(50) NOT NULL)")
 print("ElectricVehicleData table is created....")



You're connected to database.
Creating table....
ElectricVehicleData table is created....


In [1]:
#Inserting values of csv file into newly created table ElectricVehicleData

import mysql.connector
import csv
conn = mysql.connector.connect( host="localhost",database="X22114386Project",user="root",password="Caroline@5")

with open('ElectricVehdata.csv', 'r') as f:

 reader = csv.reader(f)
 next(reader) # skip the header row
 for row in reader:
  # Insert each row into the database
  cursor = conn.cursor()
  
  #Inserting values into the created table
  cursor.execute("INSERT INTO ElectricVehicleData (VIN, County, City, State, Postal_Code, Model_Year, Make, Model, Electric_Vehicle_Type, Clean_Alternative_Fuel_Vehicle_Eligibility, Electric_Range, Base_MSRP, Legislative_District, DOL_Vehicle_ID, Vehicle_Location, Electric_Utility, Census_Tract, Counties, Congressional_Districts, WAOFM_GIS) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19]))
  conn.commit()

#Closing the database connection

conn.close()


In [5]:
#Installing the module to connect to postgres

pip install psycopg2


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.2/1.2 MB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Creating the Database in postgresssql

import psycopg2

try:
 dbConnection = psycopg2.connect(user = "dap", password = "dap", host = "87.44.4.70", port = "5432", database = "postgres")
 
 dbConnection.set_isolation_level(0) # AUTOCOMMIT
 dbCursor = dbConnection.cursor()
    
 dbCursor.execute('CREATE DATABASE postgres;')
 dbCursor.close()
    
except (Exception, psycopg2.Error) as dbError :
 print ("Error while connecting to PostgreSQL", dbError)

finally:
 if(dbConnection): dbConnection.close()

Error while connecting to PostgreSQL database "postgres" already exists



In [2]:
#Creating a table in postgres

import psycopg2

conn = psycopg2.connect(database="postgres", user="dap", password="dap", host="87.44.4.70", port="5432")

cur = conn.cursor()
 
cur.execute('DROP TABLE IF EXISTS ElectVehicleData;')  
print('Creating table....')
cur.execute("""
    CREATE TABLE ElectVehicleData (
        VIN VARCHAR(50), County VARCHAR(50) NOT NULL, City VARCHAR(50) NOT NULL, State VARCHAR(50) NOT NULL, Postal_Code VARCHAR(50), Model_Year VARCHAR(50), Make VARCHAR(50), Model VARCHAR(50), Electric_Vehicle_Type VARCHAR(100), Clean_Alternative_Fuel_Vehicle_Eligibility VARCHAR(100), Electric_Range VARCHAR(50), Base_MSRP VARCHAR(50),
        Legislative_District VARCHAR(50), DOL_Vehicle_ID VARCHAR(50), Vehicle_Location VARCHAR(50), Electric_Utility VARCHAR(250), 
        Census_Tract VARCHAR(50), Counties VARCHAR(50), Congressional_Districts VARCHAR(50), WAOFM_GIS VARCHAR(50));""")

print('ElectVehicleData table created.')
# Commit the transaction and close the cursor and connection
conn.commit()
cur.close()
conn.close()


Creating table....
ElectVehicleData table created.


In [3]:
#Inserting values into table in postgres

import psycopg2
import csv

conn = psycopg2.connect(database="postgres", user="dap", password="dap", host="87.44.4.70", port="5432")

cur = conn.cursor()

insert_statement = "INSERT INTO ElectVehicleData (VIN, County, City, State, Postal_Code, Model_Year, Make, Model, Electric_Vehicle_Type, Clean_Alternative_Fuel_Vehicle_Eligibility, Electric_Range, Base_MSRP, Legislative_District, DOL_Vehicle_ID, Vehicle_Location, Electric_Utility, Census_Tract, Counties, Congressional_Districts, WAOFM_GIS) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s)"

# Open the CSV file
with open('ElectricVehdata.csv', 'r') as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')

    next(csv_reader)

    for col in csv_reader:

        values_to_insert = (col[0], col[1], col[2], col[3], col[4], col[5], col[6], col[7], col[8], col[9], col[10], col[11], col[12], col[13], col[14], col[15], col[16], col[17], col[18], col[19])
        
        # Executing the INSERT statement
        cur.execute(insert_statement, values_to_insert)
        
print('Values are inserted into the table.')
# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


Values are inserted into the table.
